In [ ]:

!pip install gfpgan basicsr facexlib scikit-image opencv-python


In [ ]:

import cv2
import torch
import numpy as np
from gfpgan import GFPGANer
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from google.colab import files
import matplotlib.pyplot as plt


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)


In [ ]:

restorer = GFPGANer(
    model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth',
    upscale=2,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None,
    device=device
)
restorer.gfpgan.eval()


In [ ]:

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [ ]:

cropped_faces, restored_faces, restored_img = restorer.enhance(
    img,
    has_aligned=False,
    only_center_face=False,
    paste_back=True
)


In [ ]:

restored_img = cv2.resize(restored_img, (img.shape[1], img.shape[0]))

img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
restored_gray = cv2.cvtColor(restored_img, cv2.COLOR_RGB2GRAY)

psnr_value = psnr(img_gray, restored_gray)
ssim_value = ssim(img_gray, restored_gray)

print(f"PSNR : {psnr_value:.2f} dB")
print(f"SSIM : {ssim_value:.4f}")


In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Gambar Asli")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(restored_img)
plt.title("Hasil Restorasi GFPGAN")
plt.axis("off")

plt.show()
